In [1]:
debug = False

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

# from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import AgentType, AgentExecutor
from langchain.tools import StructuredTool
from langchain.chat_models import ChatOpenAI

from typing import List, Dict
import panel as pn
import param
import pandas as pd
import geopandas as gpd

from modules.chat_utils import tools, map_mgr#, chat_box
# from modules.rasterize_plots import s2_hv_plot, create_rgb_viewer

pd.options.plotting.backend = 'holoviews'

In [2]:
# this is just my little agent & panel layout/ template
chat_box = pn.widgets.ChatBox(ascending=True)

def chat(user_messages: List[Dict[str, str]]) -> None:
    # user_messages = [{"You": "Your input"}, {"AI": "A response"}, ...]
    user_message = user_messages[-1]
    input = user_message.get("You")
    if input is None:
        return
    text = agent.run(input=input)
    
    if map_mgr.media is not None:
        media = map_mgr.media
        chat_box.append({"SatGPT": media})
        map_mgr.media = None

    chat_box.append({"SatGPT": text})

pn.bind(chat, user_messages=chat_box, watch=True)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)


agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True,
    agent_kwargs=agent_kwargs, 
    memory=memory
    )
component = pn.Column(chat_box, height=800)

template = pn.template.FastListTemplate(
    # site="Awesome Panel",
    title="SatGPT",
    logo="https://panel.holoviz.org/_static/logo_stacked.png",
    main=[component],
)

if debug == False:
    template.servable()


In [3]:
if debug == True:
    pn.serve(template)

Launching server at http://localhost:36905


In [4]:
# agent.run("search sentinel images over seattle, May 2022 & load to map")

In [14]:
# agent.run("search Landsat images over seattle, May 2022")



> Entering new  chain...

Invoking: `stac_search` with `{'bbox': '-122.4361,47.4956,-122.2244,47.7341', 'dtime': '2022-05', 'collection': 'landsat-c2-l2'}`




/home/codespace/.local/lib/python3.10/site-packages/pystac_client/item_search.py:856: FutureWarning: get_all_items_as_dict() is deprecated, use item_collection_as_dict() instead.
  warnings.warn(


{'count': 11}I found 11 Landsat Level-2 images over Seattle in May 2022.

> Finished chain.


'I found 11 Landsat Level-2 images over Seattle in May 2022.'

In [15]:
# import pystac
# from odc.stac import stac_load
# from rioxarray.merge import merge_arrays

# items = pystac.ItemCollection(map_mgr.items_dict['features'])
# time = items[0].datetime.date()

# mask = [i.datetime.date() == time for i in items]
# items = [b for a, b in zip(mask, items) if a]

# raw_data = stac_load(
#     items,
#     resolution=200,
#     bands=['red','green','blue'],
#     chunks={"time": 1, "x": 2048, "y": 2048},
#     crs="EPSG:3857",
# ).to_array(dim="band")

# # TODO: add spatial merge back here (from image_plots?)

# # Spatial merge
# out_data = merge_arrays(dataarrays=[raw_data.squeeze() for t in raw_data["time"]])#.drop('time'))

In [16]:
# import pystac
# from odc.stac import stac_load
# from rioxarray.merge import merge_arrays

# items = pystac.ItemCollection(map_mgr.items_dict['features'])

In [17]:
# for a in map_mgr.items_dict['features'][0]['assets']:
#     print(a)

thumbnail
reduced_resolution_browse
mtl.json
mtl.txt
mtl.xml
ang
qa_pixel
qa_radsat
blue
green
red
nir08
swir16
swir22
atmos_opacity
cloud_qa
lwir
atran
cdist
drad
urad
trad
emis
emsd
qa


template = pn.template.FastListTemplate(
    # site="Awesome Panel",
    title="SatGPT",
    logo="https://panel.holoviz.org/_static/logo_stacked.png",
    header_background=ACCENT_BASE_COLOR,
    accent_base_color=ACCENT_BASE_COLOR,
    main=[component],
).servable()